In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = []
                list_div = html.xpath('//div[@class="product info detailed"]/div/div')
                for i in range(len(list_div)):
                    if len(list_div[i].xpath('./a/text()')) != 0 and list_div[i].xpath('./a/text()')[0].strip() == 'Cross Reference':
                        list_tr = list_div[i+1].xpath('./table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Url': input_.loc[a, 'Url']}])

                for tr in list_tr:
                    df_temp.loc[0, 'OE_' + tr.xpath('./td[1]/text()')[0].strip()] = tr.xpath('./td[2]/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./oe-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./oe_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1266

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2236-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1246] - [当前时间：10:30:58]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2039-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：1245] - [当前时间：10:30:58]

[ok] - https://portal.thepartshq.com/mrc-a2299-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：1244] - [当前时间：10:30:58]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2305-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1243] - [当前时间：10:30:58]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2293-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：1242] - [当前时间：10:30:59]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2268-manual-transmission-mount.html
[尝试次数：2] - [剩余数量：1241] - [当前时间：10:30:59]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2107-automatic-transmission-mount.html
[尝试次数：3] - [剩余数量：1240] - [当前时间：10:30:59]

[ok] - https://portal.thepartshq.com/marmo

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2648-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1186] - [当前时间：10:31:23]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2639-automatic-transmission-mount.html
[尝试次数：3] - [剩余数量：1185] - [当前时间：10:31:23]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2497-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1184] - [当前时间：10:31:25]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2664-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1183] - [当前时间：10:31:26]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2657-automatic-transmission-mount.html
[尝试次数：5] - [剩余数量：1182] - [当前时间：10:31:26]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2513-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1181] - [当前时间：10:31:28]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2679-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1180] - [当前时间：10:31:28]

[ok] - https://portal.thepa

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2882-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1127] - [当前时间：10:32:09]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2871-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1126] - [当前时间：10:32:12]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2887-automatic-transmission-mount-bushing.html
[尝试次数：1] - [剩余数量：1125] - [当前时间：10:32:13]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2525-automatic-transmission-mount-bushing.html
[尝试次数：3] - [剩余数量：1124] - [当前时间：10:32:14]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2163-manual-transmission-mount.html
[尝试次数：3] - [剩余数量：1123] - [当前时间：10:32:14]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2880hy-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1122] - [当前时间：10:32:14]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2896hy-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1121] - [当前时间：10:32:15]

[ok] - htt

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2864-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：1067] - [当前时间：10:32:45]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2873-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1066] - [当前时间：10:32:45]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2881-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：1065] - [当前时间：10:32:46]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a42000-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1064] - [当前时间：10:32:46]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4088-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1063] - [当前时间：10:32:47]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4074-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1062] - [当前时间：10:32:47]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4091-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1061] - [当前时间：10:32:47]

[ok] - https://portal.thep

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4422-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：1007] - [当前时间：10:33:17]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4423-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1006] - [当前时间：10:33:17]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4409-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：1005] - [当前时间：10:33:18]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4430-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：1004] - [当前时间：10:33:28]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a4312-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：1003] - [当前时间：10:33:28]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a42058-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：1002] - [当前时间：10:33:28]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a42021-manual-transmission-mount.html
[尝试次数：2] - [剩余数量：1001] - [当前时间：10:33:29]

[ok] - https://portal.thepartsh

...

[ok] - https://portal.thepartshq.com/marmon-ride-control-am153-manual-transmission-mount.html
[尝试次数：3] - [剩余数量：280] - [当前时间：10:40:36]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2124-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：279] - [当前时间：10:40:38]

[ok] - https://portal.thepartshq.com/marmon-ride-control-am283-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：278] - [当前时间：10:40:38]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2127-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：277] - [当前时间：10:40:40]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2121-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：276] - [当前时间：10:40:41]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2119-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：275] - [当前时间：10:40:42]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2014-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：274] - [当前时间：10:40:43]

[ok] - https://portal.thepartshq.com/mar

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2771-manual-transmission-mount.html
[尝试次数：2] - [剩余数量：220] - [当前时间：10:41:20]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2638hd-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：219] - [当前时间：10:41:21]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2854-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：218] - [当前时间：10:41:22]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2824hy-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：217] - [当前时间：10:41:23]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2821hy-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：216] - [当前时间：10:41:23]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2769-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：215] - [当前时间：10:41:24]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a2730-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：214] - [当前时间：10:41:24]

[ok] - https://portal.thepartshq.com/

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5403-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：160] - [当前时间：10:41:56]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5401-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：159] - [当前时间：10:41:57]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5405-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：158] - [当前时间：10:41:57]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5101-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：157] - [当前时间：10:41:58]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5427-automatic-transmission-mount-bushing.html
[尝试次数：1] - [剩余数量：156] - [当前时间：10:41:59]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5233-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：155] - [当前时间：10:41:59]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5472-automatic-transmission-mount-bushing.html
[尝试次数：1] - [剩余数量：154] - [当前时间：10:42:00]

[ok] - https://portal

[ok] - https://portal.thepartshq.com/marmon-ride-control-a65046-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：99] - [当前时间：10:42:38]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a6285-automatic-transmission-mount.html
[尝试次数：2] - [剩余数量：98] - [当前时间：10:42:39]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a6517-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：97] - [当前时间：10:42:40]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a6518-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：96] - [当前时间：10:42:40]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a6546-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：95] - [当前时间：10:42:41]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a65035-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：94] - [当前时间：10:42:41]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a6550-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：93] - [当前时间：10:42:41]

[ok] - https://portal.thepartshq.com/marmon-r

[ok] - https://portal.thepartshq.com/marmon-ride-control-a71083-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：39] - [当前时间：10:43:19]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a7201-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：38] - [当前时间：10:43:19]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a7226-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：37] - [当前时间：10:43:19]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a5208-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：36] - [当前时间：10:43:19]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a7248-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：35] - [当前时间：10:43:21]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a7258-automatic-transmission-mount.html
[尝试次数：1] - [剩余数量：34] - [当前时间：10:43:21]

[ok] - https://portal.thepartshq.com/marmon-ride-control-a7247-manual-transmission-mount.html
[尝试次数：1] - [剩余数量：33] - [当前时间：10:43:21]

[ok] - https://portal.thepartshq.com/marmon-ride-